In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

import re  
import nltk

from nltk.corpus import stopwords
from nltk.corpus import RegexpTokenizer as regextoken
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import pickle

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.decomposition import PCA
from __future__ import print_function

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb


In [32]:
#Import the saved train data
FnllBanklvldata_Tgt = pd.read_pickle('FnllBanklvldata_Tgt_Saved.pkl')
#FnllBanklvldata_Tgt.to_csv('Banktrain.csv')

In [33]:
#Distribution of bank grade
Bankgrade = FnllBanklvldata_Tgt[['BankGrade','BankID']].groupby(['BankGrade'])['BankID'].nunique().reset_index(name='Count')
Bankgrade

,BankGrade,Count
0,deficient,18
1,outstanding,14
2,satisfactory,54


In [34]:
#Import the saved test data
TstFnllBanklvldata_Tgt = pd.read_pickle('TestFnllBanklvldata_Tgt_Saved.pkl')
#TstFnllBanklvldata_Tgt.to_csv('Banktest.csv')

In [1393]:
#dropc = ['ClosExpl_pct','ClosExplDispCom_pct','MortgageBroad_pct','MediandaystoResolve','Lessthan1Month_pct','onetotwoMonths_pct','BankID','State53_pct','twotothreeMonths_pct','State14_pct','Communicn_pct','purchaser_pct','State47_pct','threeormoreMonths_pct']
#Modeldata=FnllBanklvldata_Tgt.drop(dropc, axis=1)
#Testdata=TstFnllBanklvldata_Tgt.drop(dropc, axis=1)

In [1379]:
#without PCA -> these set of independent variables are not correlated to each other
#Modeldata1 = Modeldata[['BankGrade','AvgdaystoResolve','State43_pct','LateFeeIntCharg_pct','TeftFraudThreat_pct','ClosExplDisp_pct']]
#Testdata1 = Testdata[['AvgdaystoResolve','State43_pct','LateFeeIntCharg_pct','TeftFraudThreat_pct','ClosExplDisp_pct']]

In [ ]:
#use for without PCA
#X = Modeldata1.iloc[:, [1,2,3,4,5]].values
#y = Modeldata1.iloc[:, 0].values
#Test=Testdata1.iloc[:, [0,1,2,3,4]].values

In [35]:
#for PCA
Modeldata1 = FnllBanklvldata_Tgt[['BankGrade','AvgdaystoResolve','State43_pct','LateFeeIntCharg_pct','CloseCancel_pct','MktOfferRwd_pct','TeftFraudThreat_pct','CreditCard_pct','UFDP_pct','Payments_pct','ClosExplDisp_pct']]
Testdata1 = TstFnllBanklvldata_Tgt[['AvgdaystoResolve','State43_pct','LateFeeIntCharg_pct','CloseCancel_pct','MktOfferRwd_pct','TeftFraudThreat_pct','CreditCard_pct','UFDP_pct','Payments_pct','ClosExplDisp_pct']]
Testdata1.head(1)


,AvgdaystoResolve,State43_pct,LateFeeIntCharg_pct,CloseCancel_pct,MktOfferRwd_pct,TeftFraudThreat_pct,CreditCard_pct,UFDP_pct,Payments_pct,ClosExplDisp_pct
0,17.49343,12.16,26.08,36.93,44.42,19.65,99.47,64.45,11.83,27.27


In [36]:
#For PCA 
X = Modeldata1.iloc[:, [1,2,3,4,5,6,7,8,9,10]].values
y = Modeldata1.iloc[:, 0].values
Test=Testdata1.iloc[:, [0,1,2,3,4,5,6,7,8,9]].values

## SMOTE - for class imbalance

In [37]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X, y = smt.fit_sample(X, y)

In [38]:
# Splitting the dataset into the Training set and val set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [1382]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X = sc.fit_transform(X)
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [1383]:
X.shape

(162, 5)

In [810]:
#PCA
from sklearn.decomposition import PCA
    
pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
Test=pca.transform(Test)
X = pca.fit_transform(X)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

output variance explained by the pricipal components along with feature matrix
print ("Variance explained by all the principal components = ", sum(explained_variance))

In [1009]:
#Kernel PCA

#from sklearn.decomposition import KernelPCA

#kpca=KernelPCA(n_components=4,kernel='rbf')
#X_train = kpca.fit_transform(X_train)
#X_val = kpca.transform(X_val)
#Test=kpca.transform(Test)
#X = kpca.fit_transform(X)

## 1) Logistic Regression 

#### Fitting Logistic Regression to the Training set

In [1252]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0,solver='lbfgs',multi_class='multinomial')

In [1253]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[47  0  7]
 [ 0 48  6]
 [ 9  7 38]]
0.8209876543209876


#### K fold validation

In [1249]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())


K fold accuracy vector: [0.88888889 0.77777778 0.61111111 0.83333333 0.86666667 0.66666667
 0.8        0.8        0.73333333 0.8       ]

K fold accuracy mean: 0.7777777777777778

K fold accuracy standard deviation: 0.0818007214191376


In [1250]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[41  1  3]
 [ 0 38  4]
 [ 9  5 28]]
0.8294573643410853
[[ 8  0  1]
 [ 0 10  2]
 [ 3  3  6]]
0.7272727272727273


In [1254]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient'], dtype=object)

#### Export the test predictions to csv

In [1158]:
predictions=pd.DataFrame(predictions)
BankID=  TstFnllBanklvldata_Tgt[['BankID']]
frames = [BankID,predictions]
TestPredictions = pd.concat(frames,axis=1)
TestPredictions.to_csv('TestPredictions.csv')

## 2) Naive Bayes Model

In [1260]:
from sklearn.naive_bayes import GaussianNB

# Fitting classifier to the Training set
classifier=GaussianNB()
classifier

GaussianNB(priors=None)

In [1261]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[44  0 10]
 [ 0 49  5]
 [14 10 30]]
0.7592592592592593


In [1257]:
#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

K fold accuracy vector: [0.77777778 0.77777778 0.55555556 0.83333333 0.73333333 0.66666667
 0.8        0.73333333 0.93333333 0.73333333]

K fold accuracy mean: 0.7544444444444445

K fold accuracy standard deviation: 0.09486182283773999


In [1258]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[39  0  6]
 [ 1 38  3]
 [12  8 22]]
0.7674418604651163
[[ 5  0  4]
 [ 0 11  1]
 [ 3  2  7]]
0.696969696969697


In [1262]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory'], dtype='|S12')

## 3) SVM Model

In [1269]:
from sklearn.svm import SVC

#We can also pecifiy the C parameter 
classifier=SVC(kernel='linear',random_state=0)
classifier

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [1270]:
classifier=SVC(C=1,kernel='linear',random_state=0)

In [1271]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[46  0  8]
 [ 0 46  8]
 [ 8  7 39]]
0.808641975308642


In [1266]:
#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

K fold accuracy vector: [0.77777778 0.72222222 0.66666667 0.88888889 0.86666667 0.66666667
 0.8        0.73333333 0.8        0.8       ]

K fold accuracy mean: 0.7722222222222221

K fold accuracy standard deviation: 0.07153519379031055


In [1267]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[38  0  7]
 [ 0 39  3]
 [ 8  5 29]]
0.8217054263565892
[[ 8  0  1]
 [ 0 10  2]
 [ 2  3  7]]
0.7575757575757576


In [1272]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient'], dtype=object)

## 4) SVM Kernel

In [1281]:
from sklearn.svm import SVC

#We can also pecifiy the C parameter 
classifier=SVC(kernel='rbf',random_state=0)
classifier

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [1282]:
#classifier=SVC(C=1000,gamma=0.001,kernel='rbf',random_state=0)

#Kernel PCA
classifier=SVC(C=10,gamma=0.5,kernel='rbf',random_state=0)
#classifier

In [1283]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[54  0  0]
 [ 0 54  0]
 [ 0  0 54]]
1.0


In [1276]:
#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

K fold accuracy vector: [0.72222222 0.83333333 0.88888889 0.77777778 1.         0.8
 0.93333333 1.         0.86666667 0.86666667]

K fold accuracy mean: 0.868888888888889

K fold accuracy standard deviation: 0.08646700648152765


## Grid search / parameter tuning 

In [1277]:
from sklearn.model_selection import GridSearchCV
parameters=[{'C':[1,10,100,1000],'kernel':['linear']},{'C':[1,10,100,1000],'kernel':['rbf'],'gamma':[0.5,0.1,0.01,0.001,0.0001]}]
grid_search=GridSearchCV(estimator=classifier,param_grid=parameters,scoring='accuracy',cv=10)

grid_search

grid_search=grid_search.fit(X,y)
best_accuracy=grid_search.best_score_
best_parameters=grid_search.best_params_
print("Best accuracy: ",best_accuracy)
print("Best parameters: ",best_parameters)

Best accuracy:  0.8641975308641975
Best parameters:  {'kernel': 'rbf', 'C': 10, 'gamma': 0.5}


In [1278]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[45  0  0]
 [ 0 42  0]
 [ 0  0 42]]
1.0
[[ 9  0  0]
 [ 0 11  1]
 [ 4  1  7]]
0.8181818181818182


In [1284]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory', 'satisfactory', 'satisfactory',
       'satisfactory', 'satisfactory'], dtype=object)

In [1280]:
#Export predictions to excel

predictions=pd.DataFrame(predictions)
BankID=  TstFnllBanklvldata_Tgt[['BankID']]
frames = [BankID,predictions]
TestPredictions = pd.concat(frames,axis=1)
TestPredictions.to_csv('TestPredictions_SVMKernel_rbf1000_001.csv')

## 5) KNN Model

In [1290]:
from sklearn.neighbors import KNeighborsClassifier

#We can also pecifiy the C parameter 
classifier=KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2) #Euclidean distance
classifier

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [1291]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[54  0  0]
 [ 1 53  0]
 [11  7 36]]
0.8827160493827161


In [1292]:
#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

K fold accuracy vector: [0.66666667 0.77777778 0.5        0.72222222 0.86666667 0.66666667
 0.8        1.         0.66666667 0.8       ]

K fold accuracy mean: 0.7466666666666667

K fold accuracy standard deviation: 0.12879306782970756


In [1288]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[43  1  1]
 [ 1 41  0]
 [10  6 26]]
0.8527131782945736
[[ 8  1  0]
 [ 0 12  0]
 [ 7  3  2]]
0.6666666666666666


In [1293]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['deficient', 'deficient', 'deficient', 'satisfactory', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient'], dtype=object)

## 6) Decision Tree Classification

In [1294]:
from sklearn.tree import DecisionTreeClassifier

#We can also pecifiy the C parameter 
classifier=DecisionTreeClassifier(criterion='entropy',random_state=0) #Euclidean distance
classifier

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [1295]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[54  0  0]
 [ 0 54  0]
 [ 0  0 54]]
1.0


In [1296]:
#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

K fold accuracy vector: [0.83333333 0.88888889 0.77777778 0.72222222 0.93333333 0.86666667
 0.86666667 0.86666667 0.53333333 0.66666667]

K fold accuracy mean: 0.7955555555555556

K fold accuracy standard deviation: 0.11634346766851934


In [1297]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[45  0  0]
 [ 0 42  0]
 [ 0  0 42]]
1.0
[[ 8  0  1]
 [ 0 11  1]
 [ 3  0  9]]
0.8484848484848485


In [1298]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient', 'deficient', 'deficient', 'deficient',
       'deficient', 'deficient'], dtype=object)

## 7)Random Forest Classification

In [1299]:
from sklearn.ensemble import RandomForestClassifier

#We can also pecifiy the C parameter 
classifier=RandomForestClassifier(n_estimators=5,criterion='entropy',random_state=0) #Euclidean distance
classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [1300]:
#Fit on full train data
classifier.fit(X, y)

#Predicting the train set results
y_pred = classifier.predict(X)

#Making the Confusion Matrix for train
cmtrainfull = confusion_matrix(y, y_pred)
print(cmtrainfull)
print(accuracy_score(y, y_pred))

[[54  0  0]
 [ 0 54  0]
 [ 1  2 51]]
0.9814814814814815


In [1301]:
#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X,y=y,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

K fold accuracy vector: [0.66666667 0.77777778 0.72222222 0.88888889 0.8        0.86666667
 0.93333333 0.86666667 0.66666667 0.8       ]

K fold accuracy mean: 0.7988888888888889

K fold accuracy standard deviation: 0.08755245341374183


In [40]:
#Fit on 80% train data
classifier.fit(X_train, y_train)

#Predicting the train set results
y_pred = classifier.predict(X_train)

# Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, y_pred)
print(cmtrain)
print(accuracy_score(y_train, y_pred))

#Predicting the val set results
y_pred = classifier.predict(X_val)

# Making the Confusion Matrix for val
cmval = confusion_matrix(y_val, y_pred)
print(cmval)
print(accuracy_score(y_val, y_pred))

[[45  0  0]
 [ 0 42  0]
 [ 0  0 42]]
1.0
[[ 8  0  1]
 [ 0 11  1]
 [ 2  1  9]]
0.8484848484848485


In [41]:
# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

array(['deficient', 'deficient', 'satisfactory', 'deficient', 'deficient',
       'satisfactory', 'satisfactory', 'satisfactory', 'outstanding',
       'satisfactory', 'satisfactory', 'deficient', 'satisfactory',
       'satisfactory', 'satisfactory', 'deficient', 'satisfactory',
       'satisfactory', 'deficient', 'satisfactory', 'satisfactory',
       'satisfactory'], dtype=object)

## 8) XG Boost Model

In [42]:
X_train, X_val, y_train, y_val

# Fitting XGBoost to the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

#Predicting the train set results
ytrain_pred = classifier.predict(X_train)
yval_pred = classifier.predict(X_val)

#Making the Confusion Matrix for train
cmtrain = confusion_matrix(y_train, ytrain_pred)
cmval = confusion_matrix(y_val, yval_pred)

print(cmtrain)
print(accuracy_score(y_train, ytrain_pred))

print(cmval)
print(accuracy_score(y_val, yval_pred))


#K fold cross valdation

from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=10)

print("K fold accuracy vector:", accuracies)
print()
print("K fold accuracy mean:",accuracies.mean())
print()
print("K fold accuracy standard deviation:",accuracies.std())

# Predicting the Test set results
predictions = classifier.predict(Test)
predictions

[[45  0  0]
 [ 0 42  0]
 [ 0  0 42]]
1.0
[[ 8  0  1]
 [ 0 11  1]
 [ 2  1  9]]
0.8484848484848485
K fold accuracy vector: [0.8        0.6        0.84615385 0.69230769 0.69230769 0.83333333
 0.75       0.91666667 0.91666667 1.        ]

K fold accuracy mean: 0.8047435897435896

K fold accuracy standard deviation: 0.11659063456545757


array(['deficient', 'deficient', 'satisfactory', 'deficient', 'deficient',
       'satisfactory', 'satisfactory', 'satisfactory', 'outstanding',
       'satisfactory', 'satisfactory', 'deficient', 'satisfactory',
       'satisfactory', 'satisfactory', 'deficient', 'satisfactory',
       'satisfactory', 'deficient', 'satisfactory', 'satisfactory',
       'satisfactory'], dtype=object)

In [1403]:
predictions=pd.DataFrame(predictions)
BankID=  TstFnllBanklvldata_Tgt[['BankID']]
frames = [BankID,predictions]
TestPredictions = pd.concat(frames,axis=1)
TestPredictions.to_csv('TestPredictions_XGBoostAllSmote.csv')

array(['deficient', 'deficient', 'satisfactory', 'satisfactory',
       'deficient', 'satisfactory', 'satisfactory', 'outstanding',
       'outstanding', 'satisfactory', 'satisfactory', 'deficient',
       'satisfactory', 'satisfactory', 'satisfactory', 'deficient',
       'satisfactory', 'satisfactory', 'deficient', 'outstanding',
       'satisfactory', 'satisfactory'], dtype=object)

In [44]:
sorted_idx = np.argsort(classifier.feature_importances_)[::-1]

In [45]:
sorted_idx

array([9, 0, 7, 2, 1, 8, 4, 5, 3, 6])